## 코드 변경 사항

1. setting: synonyms_filter 설정값 추가
2. mapping: 
    - "similarity": "lm_jelinek_mercer"
    - embeddings 수정
3. Function calling:
    - "required": ["standalone_query"] -> "required": ["long_question"] 변경
4. def hybrid_retrieve 생성
5. sentence_transformers finetuning 모델 사용 -> 성능 하락
6. sentence_transformers 모델 jhgan/ko-sroberta-multitask 변경
7. topk=1 and topk=5 로 변경

In [1]:
import os
import json
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer, models

from transformers import AutoTokenizer, AutoModel
import torch

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 검색엔진 준비

In [2]:
es_username = "elastic"
es_password = "your_password"

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/data/ephemeral/home/elasticsearch-8.15.2/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

{'name': 'instance-12422', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'iMiWgoTFSGiYX_b6U44nxA', 'version': {'number': '8.15.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3', 'build_date': '2024-09-19T10:06:03.564235954Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [3]:
# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("jhgan/ko-sroberta-multitask") #ddobokki/klue-roberta-base-nli-sts

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
# SentenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100): 
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size, retrieved_docids=None):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 100
    }

    if retrieved_docids: ## 차이
        print("retrieved docids: ", retrieved_docids)
        knn = {
            "field": "embeddings",
            "query_vector": query_embedding.tolist(),
            "k": size,
            "num_candidates": 100,
            "filter": {
                "terms": {"docid": retrieved_docids}
            }
        }
    
    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)

# 역색인 + Vector 유사도 혼합
def hybrid_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    body = {
        "query": {
            "match": {
                "content": {
                    "query": query_str,
                    "boost" : 0.0025
                }
            }
        },
        "knn": {
            "field": "embeddings",
            "query_vector": query_embedding.tolist(),
            "k": 5,
            "num_candidates": 50,
            "boost": 1
        },
        "size": size
    }
    
    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", body=body)

In [5]:
# 색인을 위한 setting 설정
settings = {
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed", 
                # "filter": ["nori_posfilter"]
                "filter": ["nori_posfilter", "synonyms_filter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX", "XSA"]
                            # ["IC", "MAG", "MAJ", "MM", "SP", "SSC", "SSO", "XPN", "XSA", "XSN", "XSV", "UNA", "NA", "VSV" ] 
            },
            "synonyms_filter": {
                "type": "synonym",
                "lenient": True,
                "synonyms": [
                    "복숭아, 복숭아나무",
                    "밀물, 만조",
                    "썰물, 간조",
                    "관계, 서로 연결, 긴밀",
                    "순기능, 도움",
                    "리보오솜, 리보솜",
                    "네트웍, 네트워크",
                    "역할, 기능, 기여, 담당",
                    "전력, 전류",
                    "이루어져, 구성, 형성",
                    "땅, 토양",
                    "가장 많이, 대부분",
                    "단점, 부정적인",
                    "원자력 발전, 에너지",
                    "형태, 유형",
                    "원자, 가장 작은 입자",
                    "내부 구조, 구성"
                    "자연, 생태계",
                    "뭉쳐, 모여",
                    "날씨, 기후",
                    "측정, 추정",
                    "비만도, 체중 상태",
                    "관계, 영향",
                    "뭐야, 의미",
                    "해안, 바다",
                    "전구, 램프",
                    "번식, 생식",
                    "작은 기체 하나, 기체의 분자",
                    "다음 세대, 후손",
                    "무게, 질량",
                    "잠, 수면",
                    "약, 약물",
                    "자석, 자성체",
                    "동물, 생물체",
                    "누가, 비해",
                    "결혼 전, 혼전",
                    "숨, 호흡",
                    "인터페론, interferon",
                    "트로이군, trojan",
                    "슈퍼옥시드 디스무타아제, Superoxide Dismutase",
                    "드미트리 이바노프스키, Dmitri Ivanovsky",
                    "카탈로그, catalog",
                    "파이썬, python",
                    "디엔에이, DNA",
                    "이란 콘트라, 이란-콘트라",
                    "병합 정렬, merge sort",
                    "샘플, sample",
                    "연산자, operator",
                    "칼리시바이러스,칼리시 바이러스",
                    "브리지 인버터, bridge inverter",
                    "기억 상실증, 기억 상실, 기억력 저하",
                    "음파, 파",
                    "통학 버스, 학교 버스",
                    "극대화, 가장 큰",
                    "태양광을 에너지로 변환하는 능력, 광합성",
                    "자라다, 자랄",
                    "스모그, 매연",
                    "동일한, 하나의",
                    "나이가 들수록, 노인들의",
                    "학습된, 사회화",
                    "탄소, 탄소 원자",
                    "탄소의 내부 구조, 탄소 원자의 구조",
                    "생존의 확률, 생존률",
                    "원자번호, 양성자의 수, 양성자",
                    "색, 색상, 색깔"
                ]
            }
        }
    },
    "index": {
        "similarity": {
            "lm_jelinek_mercer": { 
                "type": "LMJelinekMercer", 
                "lambda": 0.7
            } 
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        "content": {
            "type": "text", 
            "analyzer": "nori",
            "similarity": "lm_jelinek_mercer"
        }, 
        "embeddings": {
            "type": "dense_vector",
            "dims": 768,
            # "dims": 1024,
            "index": True,
            "similarity": "l2_norm"
            # "similarity": "cosine"
        }
    }
}

In [6]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("/data/ephemeral/home/data/documents.jsonl") as f: #/data/ephemeral/home/data/aug_data.jsonl
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)
                
# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)

batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200
(4272, [])


In [7]:
len(index_docs[0]['embeddings'])

768

In [8]:
test_query = "동물들이 피부나 털의 색깔을 통해 생존의 확률을 높이는 방식에 대해 설명해줘." #"저항이 2배로 증가하면 전력은 어떻게 되나?"

# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3) # topk = 3
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source'])

# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)
print('-'*10)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source'])
    
# hybrid 검색 예제
search_result_retrieve = hybrid_retrieve(test_query, 3)
print('-'*10)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source'])

score: 14.614903 source: {'docid': 'dc398ded-03b2-4113-8ae6-9161cb5a545c', 'src': 'ko_ai2_arc__ARC_Challenge__test', 'content': '털은 몸을 따뜻하게 유지하는 데 도움을 주는 중요한 부분입니다. 털은 동물의 피부에 자라는 모낭으로 구성되어 있습니다. 털은 몸 온도를 조절하는 역할을 하며, 추운 날씨에는 몸을 보호하여 열을 유지하고, 더운 날씨에는 열을 흡수하여 몸을 시원하게 유지합니다. 또한, 털은 동물의 외형을 보호하고, 적응색을 제공하여 동물이 환경에 잘 적응할 수 있도록 도와줍니다. 털은 다양한 동물들에게 존재하며, 각각의 종마다 털의 형태와 기능이 다를 수 있습니다. 털은 동물들에게 매우 중요한 생리적인 기능을 제공하며, 동물들의 생존과 번식에도 영향을 미칩니다.', 'embeddings': [-0.19064626097679138, -0.21818336844444275, -0.13039232790470123, 0.012476530857384205, -0.11346571147441864, 0.23742571473121643, 0.14986246824264526, 0.2396954596042633, -0.46460944414138794, 0.2564418911933899, 0.208185076713562, -0.48138681054115295, 0.2732871174812317, 0.5477936267852783, 0.10972202569246292, 0.2602173388004303, -0.48087823390960693, 0.5085950493812561, 0.04698638990521431, -0.15978911519050598, -0.20209310948848724, 0.07249058783054352, 0.40592727065086365, 0.09174023568630219, -0.10940416157245636, -0.0463384762406349

In [9]:
test_query = "바람이 부는 이유는?" #"저항이 2배로 증가하면 전력은 어떻게 되나?"

# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3) # topk = 3
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source'])

# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)
print('-'*10)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source'])
    
# hybrid 검색 예제
search_result_retrieve = hybrid_retrieve(test_query, 3)
print('-'*10)
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source'])

score: 9.99551 source: {'docid': 'bf04bb4d-16d5-446e-b672-70e5a66b894a', 'src': 'ko_ai2_arc__ARC_Challenge__test', 'content': '바다에서 육지로 부는 바람은 육지 위에서 부는 바람보다 습도가 높습니다. 이렇게 높은 습도는 해안 지역의 기후에 다양한 영향을 미칩니다. 대표적으로는 높은 습도로 인해 해안 지역은 강수량이 더 많습니다. 바다에서 육지로 부는 바람은 바다 위에서 수증기를 흡수하고, 이를 육지로 옮겨줍니다. 따라서, 해안 지역은 더 많은 수증기를 받아들이게 되어 강수량이 증가합니다.', 'embeddings': [-0.4268876016139984, -0.5548923015594482, -0.2507362961769104, 0.33165425062179565, 0.040966738015413284, 0.16808198392391205, -0.1660526692867279, 0.229239359498024, 0.0095051359385252, 0.01734522357583046, 0.14006291329860687, -0.15497615933418274, 0.08179568499326706, -0.34448370337486267, 0.04715196043252945, -0.1569821536540985, 0.6811781525611877, 0.15982311964035034, 0.0036485022865235806, -0.3825688064098358, 0.7960385084152222, -0.011985802091658115, 0.2661076486110687, -1.0361582040786743, -0.6000298857688904, 0.2322484701871872, -0.7129907011985779, -0.1712721586227417, -0.21077488362789154, 0.30797863006591797, 0.4167550802230835, 

In [10]:
# 역색인 -> Vector 유사도 2-step 검색 예제
sparse_search_result = sparse_retrieve(test_query, 3)
        
retrieved_docids = []
for i,rst in enumerate(sparse_search_result['hits']['hits']):
    retrieved_docids.append(rst["_source"]["docid"].split('-')[0])

search_result_retrieve = dense_retrieve(test_query, 3, retrieved_docids)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source'])

retrieved docids:  ['bf04bb4d', '56de6a81', '3749f9fd']
score: 0.009081484 source: {'docid': '56de6a81-ac1c-4581-87e5-5a2d0e80be2c', 'src': 'ko_mmlu__conceptual_physics__train', 'content': '모형 비행기는 바람을 맞으며 날 때 더 느리게 날고, 바람이 뒤에서 불 때 더 빠르게 날아간다. 이는 공기 저항과 바람의 힘에 의해 결정된다. 바람을 맞으며 날 때, 비행기는 바람과 마주치기 때문에 공기 저항이 증가하고, 따라서 더 느리게 날아간다. 반대로, 바람이 뒤에서 불 때, 비행기는 바람에 도움을 받아 공기 저항이 감소하고, 따라서 더 빠르게 날아간다. 그러나 바람에 직각으로 발사될 때, 즉 옆바람이 부는 상황에서는 대지 속도가 정체 공기에서 날아갈 때와 비교하여 더 크다. 이는 바람의 힘과 비행기의 날개 형태에 의해 결정된다. 옆바람이 부는 상황에서 비행기는 바람의 힘을 받아 옆으로 밀리게 되는데, 이로 인해 비행기는 대지 속도를 더욱 증가시킬 수 있다. 따라서, 옆바람이 부는 상황에서 비행기의 대지 속도는 정체 공기에서 날아갈 때와 비교하여 더 크다.', 'embeddings': [-0.7131688594818115, -0.11493770778179169, 0.20050905644893646, 0.6625921130180359, -0.01821419596672058, 0.10748123377561569, 0.05860241502523422, 1.0165033340454102, -0.12709826231002808, 0.1118880957365036, 0.09275268018245697, -0.5570356845855713, -0.6187965869903564, -0.32100367546081543, -0.39432844519615173, -0.14735916256904602, 0.03993990644812584, 0

## RAG 구현

In [11]:
# 아래부터는 실제 RAG를 구현하는 코드입니다.
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = "your_api_key"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"  #"gpt-4o-mini-2024-07-18" #"gpt-3.5-turbo-1106"

In [12]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 지식에 관해 질문하는 경우에는 반드시 search 함수를 호출한다.
- 나머지 메시지에는 함수 호출 없이 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "long_question": {
                        "type": "string",
                        # "description": "Final query suitable for use in search from the user messages history"
                        # "description": "Keywords suitable for use in search engine."
                        # "description": "User's question in Korean, including all the keywords in the user messages"
                        "description": "User's question in Korean. Full message if the user message is single-turn."                    }
                },
                "required": ["long_question"],
                "type": "object"
            }
        }
    },
]

In [13]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""} 

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("long_question")
        print(f'standalone_query: {standalone_query}\n')
        
        # 검색 결과 추출
        # search_result = sparse_retrieve(standalone_query, 3)
        # search_result = dense_retrieve(standalone_query, 3)
        search_result = hybrid_retrieve(standalone_query, 3) # 기본 3

        # # 2-step: sparse -> dense
        # sparse_search_result = sparse_retrieve(standalone_query, 3)
        
        # retrieved_docids = []
        # for i,rst in enumerate(sparse_search_result['hits']['hits']):
        #     retrieved_docids.append(rst["_source"]["docid"].split('-')[0])

        # search_result = dense_retrieve(standalone_query, 3, retrieved_docids)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i,rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        # openai 토큰 절약을 위해 검색 결과 기반 답변 생성 생략
        # content = json.dumps(retrieved_context)
        # messages.append({"role": "assistant", "content": content})
        # msg = [{"role": "system", "content": persona_qa}] + messages
        # try:
        #     qaresult = client.chat.completions.create(
        #             model=llm_model,
        #             messages=msg,
        #             temperature=0,
        #             seed=1,
        #             timeout=30
        #         )
        # except Exception as e:
        #     traceback.print_exc()
        #     return response
        # response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response

In [14]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    pred = []
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            # print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            pred.append(output)
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1 
    return pred

In [15]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
pred = eval_rag("/data/ephemeral/home/data/eval.jsonl", "/data/ephemeral/home/result/result_final_v4.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
standalone_query: 나무의 분류에 대해 조사해 보기 위한 방법은?

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
standalone_query: 각 나라에서의 공교육 지출 현황에 대해 알려줘.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
standalone_query: 기억 상실증이 발생하는 원인은 무엇인가요?

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
standalone_query: 통학 버스의 가치에 대해 말해줘.

Test 4
Question: [{'role': 'user', 'content': 'Dmitri Ivanovsky가 누구야?'}]
standalone_query: Dmitri Ivanovsky가 누구인가요?

Test 5
Question: [{'role': 'user', 'content': '피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?'}]
standalone_query: 피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?

Test 6
Question: [{'role': 'user', 'content': '헬륨이 다른 원소들과 반응을 잘 안하는 이유는?'}]
standalone_query: 헬륨이 다른 원소들과 반응을 잘 안하는 이유는?

Test 7
Question: [{'role': 'user', 'content': '문맹 비율이 사회 발전에 미치

## 검증 스코어 계산

In [16]:
# 결과를 저장할 리스트 초기화
pred = []

with open("/data/ephemeral/home/result/result_final_v4.csv", "r") as f:
    for line in f:
        data = json.loads(line)
        pred.append(data)

# 결과 출력
print(pred)

[{'eval_id': 78, 'standalone_query': '나무의 분류에 대해 조사해 보기 위한 방법은?', 'topk': ['c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d', 'e227a022-da3b-4810-9882-a2b27c76cc79', '9712bdf6-9419-4953-a8f1-8a4015dee986'], 'answer': '', 'references': [{'score': 0.04526331, 'content': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.'}, {'score': 0.026094232, 'content': '나무는 많은 제품들이 만들어지는 소중한 자원입니다. 나무 사용을 가장 잘 관리하는 방법은 나무를 한 그루 벨 때마다 한 그루를 심는 것입니다. 이렇게 하면 나무의 자원을 지속적으로 활용할 수 있을 뿐만 아니라, 산림을 보호하고 생태계를 유지하는 데에도 도움이 됩니다. 나무를 심는 것은 우리의 미래를 위한 투자입니다. 또한, 나무를 잘 관리하기 위해서는 적절한 수분 공급과 영양분을 제공해야 합니다. 물론, 나무를 심을 때에는 적절한 위치와 조건을 고려해야 합니다. 나무는 자연의 선물이며 우리의 삶과 환경을 더욱 풍요롭게 만들어줍니다. 따라서, 나무 사용을 가장 잘 관리하기 위해서는 나무를 심고 키우는 데에 최선을 다해야 합니다.'}, {'score': 0.025210667, 'content': '생물학에서 

In [17]:
def calc_map(gt, pred):
    sum_average_precision = 0
    for j in pred:
        if gt[j["eval_id"]]: #j['eval_id'] in gt: 
            hit_count = 0
            sum_precision = 0
            for i, docid in enumerate(j["topk"][:3]):
                if docid in gt[j["eval_id"]]: 
                    hit_count += 1 
                    sum_precision += hit_count/(i+1)
            if hit_count > 0:
                average_precision = sum_precision / hit_count
            else:
                average_precision = 0
        else:
            if j["topk"]:
                average_precision = 0 
            else:
                average_precision = 1 
        sum_average_precision += average_precision 
    return sum_average_precision/len(pred)

gt = { #eval_id : topk
        78: ["c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d"],
        213: ["79c93deb-fe60-4c81-8d51-cb7400a0a156"],
        107: ["25de4ffd-cee4-4f27-907e-fd6b802c6ede"],
        81: ["bd91bda8-351e-4683-bb1a-8254f93e2376"],
        280: ["38686456-b993-4cbb-af0d-1c53df2f3e12"],
        10: ["99a07643-8479-4d34-9de8-68627854f458"],
        100: ["d9ce8432-f72e-4253-9735-98318a6f9f7f"],
        279: ["0f0dd1ae-a36c-4c97-9785-4698400c67b1", "de1ab247-9d48-48f7-8499-31606f53c108"],
        42: ["4b49f3a2-32c9-4b2e-89c4-4719f98e7a74"],
        308: ["72c780ec-57bb-4fe1-976a-d7ee3d3dbb52"],
        205: ["ae28101b-a42e-45b7-b24b-4ea0f1fb2d50"],
        289: ["1f442344-084b-44f8-838b-332be289083c", "421aac6b-49ce-4697-a68f-850152f323d7"],
        268: ["7c14c33c-f15c-41f6-ab5c-4d37c2eb3513"],
        18: ["63846d07-8443-4bf8-8cd9-bc6cc7826555"],
        9: ["bfbba89d-fdaa-400d-8848-ca1ff8d51cd7"],
        101: ["144f5e5e-8069-425f-80b3-6388195ba4ee"],
        236: ["2077ea5b-53ac-4242-bbfc-20005ad63db8"],
        59: ["82b095fd-2fb6-48ae-8476-2f457f8b6650","c1edd0df-b9f4-4dc2-90f4-38708cddddba", "4344f76e-1747-4bc9-8d02-c26db29151f4"],
        25: ["35c5dcc7-4720-4318-901e-770105ae63fd"],
        5: ["84f3f0e3-7ff2-4090-9961-aa7bbe8ca412", "59d5d7bb-6700-40ad-8884-ff43b1a9a1a0", "abf99ff1-d6bf-4020-b752-da7cb8611915"],
        104: ["73089763-06d2-4395-b235-aa3e6a399531"],
        276: [],
        14: ["2077ea5b-53ac-4242-bbfc-20005ad63db8"],
        270: ["a729b4f2-c734-4c60-9205-1518ba762593", "191c4b9f-6feb-49dd-90ad-9f2eebb6113e"],
        238: ["f48600d6-e492-43eb-b564-1860aa81da5f"],
        269: ["05b5a4f4-b115-4b76-9fe1-b80c4498289b"],
        43: ["cefe7caf-6cd1-422a-b41e-e82b543556e9", "8a78364e-63bf-4915-b718-fdc461bc62c9"],
        65: ["0bda5010-9ac6-447b-b484-60e380f4921d", "f5f54058-8c3c-4f6b-9549-db99b17685ed", "d9492876-df4d-4570-a58d-5a0438315fc9"],
        97: ["1655c90b-29c7-47ef-a092-01f2550db3aa", "85d28a10-9380-4afe-afef-b34449ef86bf"],
        206: ["70d104b2-8d74-4799-a09d-5a4c8dd577c0"],
        21: ["7150c749-dff2-4bd5-90ff-ff1e1cda468b"],
        221: ["8ae1234f-2a28-4069-a017-e99de5d67cc6"],
        71: ["5043c033-841c-46dd-94a3-1b5bef034c62"],
        254: ["af966ff7-109a-4c28-a644-393f5333ce69"],
        226: ["e21aceaf-be57-426c-b999-7ee8a309db36"],
        241: ["468d098e-2322-4950-ac11-9756f3112944"],
        261: [],
        45: ["41ca41ac-66e3-4a6b-a604-87bf8b3a8d4d"],
        19: ["d8ad7175-469b-45b5-8eb9-69504cd04f0f"],
        210: ["4764014a-4240-4c65-aa92-20eb1369a2f7"],
        231: ["5392d86a-bc7a-46c3-8272-94d982a65eed"],
        233: ["029064ed-d9f6-42cd-9b86-88cc9f611414"],
        263: ["1a277fb7-4cd7-409b-9f28-d83cef78ca10", "c8fd4323-9af9-4a0d-ab53-e563c71f9795"],
        201: ["469e37c0-a241-4675-8b1a-aa31d11a438c"],
        293: ["36788458-5fb5-4bcd-be02-3a47e5c8c19d"],
        208: ["b22a35e9-244e-44d6-b7bf-97f3ff664866"],
        282: ["2fb58e26-5ea0-4b50-b80d-4b03640042b4"],
        62: ["e7fef7f2-2549-499b-8b36-e4628119d352"],
        55: ["21383ddc-b6bb-4cf7-8815-139a3c4d9fae"],
        257: ["c8bd9b15-8ce0-4307-9f49-0f205217178f"],
        58: ["25bf6c36-116f-42c9-9d1c-c179e6292a34"],
        283: [],
        32: [],
        94: [],
        15: ["26cb5bba-0b80-41d4-9e42-aada06c879ca"],
        4: ["aa674ad5-ae70-4223-8685-e717a27dc1b3"],
        300: ["41ce1303-0091-4414-b26d-18f66101a99f"],
        243: ["5ff8f00a-a4e6-43fd-8616-3104a4c4d637"],
        34: ["55726582-8401-4a6f-889b-e9bd3953be7c", "cc6c9dc5-4d30-4653-bee7-9f3ba90fbf48", "6335780f-292d-49da-a79c-3eee1d51a903"],
        246: ["cc56ca24-fde0-458d-95f7-d3d31b79acb5", "ec539caa-4b62-4b5f-8428-489809f80611"],
        212: ["b303e4ec-87fb-40f9-8704-9037bad5af8a"],
        214: ["2213e6c8-ebb4-4cb5-b4c4-4a1773c63bcc","537715f1-138d-45c2-a8bf-65c5429f5ab9"],
        259: ["dbfa9bbf-e2da-4d01-8aea-d6f25d43ffcf"],
        267: ["c3829f80-57bf-4db6-9e06-dd81b8bb6148"],
        90: [],
        66: ["ec87a926-171d-4f62-9acc-1b870c010a16"],
        20: ["ad5d883f-4352-4d25-ba7b-cbb605c73662"],
        24: ["bf4977ff-8fa5-4e82-b957-b6955c5bcbf0", "77a60236-e5b8-4c86-a422-1f3fa9726492"],
        281: ["de5d9a1f-67f6-45ea-a4a4-0540f5b7583e"],
        264: ["31e7fe84-bd3a-4b49-8adc-cc2e1f7a5b42"],
        79: ["c59f4bbd-71f6-4073-807f-09e8f0d3efcc"],
        304: ["ff7e8f8e-8d03-419f-bd98-93e3651fd01a"],
        292: ["8eac310f-a32f-462b-8ca8-da7c8fcd41e7", "655f64b4-74e8-4f5e-9b49-13a976ad3ee4"],
        215: ["ae30b754-a275-43dc-a2c9-95ab33a7c557"],
        225: ["6fec9716-f49f-40e9-913e-db3f4df46379"],
        108: ["d23082b4-ab12-4bcb-8658-d54ba791f263"],
        98: ["5db9db05-7e10-41d4-8f1f-81259fdc8ea7"],
        38: ["d525c38e-a296-4dec-8a60-d1fb98a355e3"],
        287: ["79c61856-f978-48c5-a7a8-0863bc661106"],
        295: ["6a8852ee-b847-4bc8-9b22-a9b47b56181e"],
        255: ["62cb0474-809d-4554-82b8-861ba23d8cb4"],
        76: ["6c9bca81-11e0-41a4-b894-3bcd66dc2bf6"],
        248: ["b39fe7fd-1fa8-46ef-ba31-f4464727d56e"],
        85: ["15f317a8-1d4d-41c4-93ec-bc9be43c3984"],
        290: ["93d1aa65-a113-4dd2-8959-e8204bfea616"],
        73: ["0d316cb2-c466-4b2b-b28b-0a726d513ef2"],
        309: ["395d6c0b-a199-451e-81ed-b49bfd853927"],
        239: ["2ead4363-f521-4377-b71c-f380fd9f5094", "deb0df64-4649-4785-a87a-2ad90a819c25"],
        284: ["552fc915-5ee7-4d6f-a45f-86e8e6a5d02c", "6b11c698-eedd-4731-ae6f-c91a327c4725"],
        220: [],
        96: ["810e57ae-aab1-470d-b078-435beb1b5ce8"],
        217: ["0a29b75f-af58-4b6a-a124-0d71ffbcbc89"],
        6: ["1a707124-fe53-45aa-baaa-e3e8d6697742"],
        237: ["59d9a47e-3f46-4caa-b4e9-9d3056b3e453"],
        245: [],
        35: ["02759425-149b-467e-af41-11f924577549", "4392ecba-ea79-496b-8931-e1a79caad179", "c6f86dc0-0ecc-4232-87ad-d12dea2e4c5b"],
        242: ["209121c3-69df-414a-a1b9-c74a1b14384f"],
        296: ["2570699b-b8c3-409e-b797-a1ee57c1fd86"],
        307: ["5a8089d2-36d8-4734-a16c-85db8226bfd3"],
        229: [],
        30: ["f532c956-a335-4688-9078-b1a13a0cfe58", "cc9425b7-beaa-4bd9-9ed4-96cd82bda481"],
        16: ["21737d6c-8ec9-40fd-80ed-50548228f0e9"],
        247: [],
        285: ["b38604f4-94bd-4482-97df-7a768331558a"],
        77: ["5b8fa65f-9a69-4990-8618-50aa1911e3ec"],
        250: ["1f70c6f1-f1a7-44be-90d6-d397bace344b", "a06ea0ff-67d9-4967-bbe2-0d9022551740"],
        68: ["59ce17a4-82be-4e0c-adbd-3ef4fd4e8a33"],
        86: ["d2762c95-6397-44b5-b20e-81d4b333dd69"],
        232: ["af4a89a5-4fe3-4655-88d7-fd2fcd118441"],
        109: ["00774f87-0573-4dd8-b4dc-c43ce406a51f"],
        1: ["4a8549c8-8e81-4804-9df9-852952dd5747"],
        28: ["03390467-c6dc-4242-a039-c7e9a8bb242f"],
        203: ["f81c0a09-4082-4b6a-8546-12371ff89fba","e7b20257-29c9-4aaa-8fbf-f0a05ea67955"],
        91: ["ec326ad8-286b-4f58-9a03-31b94e969f33"],
        105: ["b7a0428e-c402-43cb-8e89-3d5f9a7f644b"],
        67: [],
        13: ["30ee5c53-1559-42f5-94c3-2ffb0b9682aa"],
        57: [],
        301: [],
        266: ["bf023a9d-47f3-4d77-8bf1-5a854b5403a7"],
        2: [],
        253: ["1c5a826a-2475-478c-b79b-eb48a59e0f5d"],
        49: ["74b5d96e-ea97-4476-850b-f78057d7c457"],
        294: ["c53b9d18-1192-4e28-843f-5ae62685a4be"],
        17: ["4d21e0e7-b334-4fe3-9eb2-bf1cbd4ef232"],
        207: ["2e0e35c2-db89-4cfd-b782-47b1acd8404d"],
        227: [],
        204: ["14eab0c3-51d8-48e0-9703-47f34f64bbd2"],
        89: ["29fade0c-58e6-4ffd-8d24-5f2fb0744790","59952996-37f3-412e-a74c-a13f0cd1aafa","6f8d2975-3a40-441e-9dd2-c555d7d2fd85"],
        224: ["7c75080c-5d8f-41e2-ad26-c90ad3f671b0"], #"f75d3717-24c8-41e6-a78d-ea6f92444209"
        256: ["f1a89f8a-7510-4400-8fbe-20bcb0a2adc3","10668a9a-0c49-4078-89ae-cade2913f3e5"],
        291: ["eb5486e9-6790-4df5-8d49-278f5eab6056"],
        8: ["95cf4ddd-4a18-4e75-bc7d-0cc10964e52e","d5569147-478a-4b93-b5f1-19dff5e4c092"],
        106: ["34965328-bc80-41a0-ba1a-daece23d4be7","fc408e3d-9c04-44c4-89e4-139cacce27e3","b2e0e809-c9e9-4465-9248-07a9b49b034f"],
        84: ["a81ef669-2b8d-41d0-a8ff-2ab5da089488","75e95fab-8764-4703-a1e0-79953943744f","daefb46c-00c4-4e12-8996-5a4134e28fc9"],
        70: ["7ec6e16f-9a7b-435e-a80d-2a1201f9f644","6d88e982-53a9-44f7-896d-d0f949a07237"],
        230: ["6dad80e2-0a4a-44ee-b6e5-748eb3033320"],
        92: ["3f2e5ed1-2522-4421-896f-86a6dbd996f5"],
        72: ["c8fd4323-9af9-4a0d-ab53-e563c71f9795"],
        211: ["a91d0f81-373c-414b-9682-8ded87f2984d"],
        258: ["627f33cc-dee7-46d1-b392-eb1a5fea3026"],
        51: ["36c3b7a5-e415-453a-a43d-f620d652b29a"],
        200: ["d1242985-4e58-41ad-bb80-86a34a6399f4"],
        274: ["5b269648-5d9f-4043-bcf5-962a8e5913e4"],
        41: ["2d1cca9c-238e-439e-ac7a-37295202ceff", "a4e26513-2c87-4acd-8675-11ccb2dcfa67"],
        87: ["c649c195-d0c2-4e1a-988b-0111661810b8","304e9cf1-a292-40d2-8b11-a936f8208ae9"],
        235: ["68ec9c78-13f8-49b9-96d9-b3040a2f4771"],
        262: ["6bc8a76c-de25-49f0-ab05-aee03b565c89"],
        288: ["9954d917-170e-477f-84c9-6d1d42927eed"],
        60: ["917b7cf4-29c9-4a12-bd8d-2421e2fd98e0"],
        222: [],
        46: ["bd5ffdb4-0bc5-41a0-a779-27b0f63fc61a"],
        234: ["7413982d-c207-4bcb-9489-13ef8c761fa4"],
        23: ["a3c79177-5bfc-4d31-a542-fa6d9cd38c22"],
        82: ["6a8b4d1c-1b9e-4f6d-8a01-8f48934ceb7d","5057d6af-55d5-4a91-9e86-832bdd14f701","48a8e51f-0ff4-479f-8b88-382f70133886"],
        61: ["e139e58c-9c3b-4b96-b393-1929ef04f553"],
        56: ["fdf2729f-f2f6-4aef-bb57-152a4215e5df"],
        83: [],
        99: ["1a4899b4-8df8-48ff-bc25-80bb75633c1e"],
        75: ["530c19d3-a054-4dea-a621-152f71873465"],
        299: ["91a2fe6f-a626-499e-8111-919e7620ca90"],
        306: ["416e57d1-d61f-4ab5-a75f-95d53f8a0fce"],
        80: ["0d7e7394-4e11-4825-8243-750088162234"],
        64: [],
        271: ["c528c66d-07cc-4fc1-976d-631b76dddc58","0598d1c1-f304-47c2-927c-6076838a69e8"],
        39: ["fc0cbdb8-0add-4931-8591-a21ecbfa5b35"],
        219: ["d41d40d7-8914-48dc-a3ee-c95afcad4f95"],
        273: ["9ae85189-ebe3-40d4-9c17-1523246e7833"],
        12: ["9b7bc48a-6e0c-450c-8364-a7da2ac4d4d4"],
        33: ["eab2a705-18b6-4bbe-84f4-acacb4cd8a1f"],
        240: ["01684b2f-9360-4d70-93b1-05adb631aa62"],
        252: ["9f51187f-8895-43a5-820f-790a9e36d49a"],
        69: ["fdb1fef4-b0ff-42c8-b326-5279ba661a90"],
        47: ["cb4a3d86-988e-4425-b295-bbe34317c5cb"],
        0: ["e85754e3-12e4-44ec-a2c4-78a5b18b8aa7"],
        52: ["70fd8635-e601-421c-9025-64fcbb2d13be"],
        103: [],
        209: ["3149730b-15c0-4e4b-a8d4-0448a477238e"],
        251: ["8b22b0a3-63f8-4b38-9d98-0af90499e212"],
        275: ["dd00abd6-626b-4741-83d1-5b36fa36a635"],
        48: ["3f5468bc-4d08-445d-8f27-13034e402d01"],
        36: ["36450671-2474-4e24-867c-46cab8f53a77"],
        218: [],
        298: ["531fef8d-f493-42bc-980f-4a86f91bc5ea"],
        40: ["464ace62-ddf2-423d-a5d7-2f17e6785c8e"],
        260: ["a71c4494-a6aa-4779-8c38-8b510453f03a","e098388d-e054-4be2-8ebe-ce680360009c","b8912c36-293d-41be-8c13-4db3b61d2a1e"],
        11: ["ec301d79-dd16-4e88-b389-eb5f12bafef8"],
        249: ["e9a3297d-f12b-4f25-94a2-78c5c2393ba7"],
        22: ["a2f73ce5-2117-4e61-b8fb-17a1c21dad37"],
        93: ["74736be6-4150-40db-93c1-a28cf671edb3"],
        31: ["8918f0f9-795d-4f15-8982-2cede58c202e","4e7af464-9ff0-499f-913c-79971d6b1c77","81762624-6b09-44e3-b5fb-973ed8f5b7de"],
        50: ["147f526c-82c9-4001-8e68-77c909b73491"],
        277: ["ba29d074-70ca-4cbc-a786-aad0870eecf1"],
        27: ["0300c15e-5e76-4b43-aead-f80319f536dd"],
        272: ["2276e958-d625-450a-8bee-b06b762e0b16"],
        54: ["39821ad7-cabe-4ccc-a6bc-4c0a9592fa4a", "26546445-53a1-41e1-85e2-7ceb0f0b3690"],
        302: ["b0f3aa6c-8e92-4044-8e53-571e38d9451c"],
        3: ["a5097fd3-23ee-40a3-81d8-2a909cc202ee","dfbf8b98-4fb7-4d72-a382-615f66b4c9ca","254558bf-c608-47aa-952e-3f94805911f3"],
        202: ["88439180-e442-4ecf-a485-88c7de0296e9"], 
        74: ["538f5608-6b05-498e-8b85-88d14723801c"],
        44: ["8ce9b365-0ab9-4230-af45-6cdfd9a4c25b"],
        63: ["157054e4-e5a1-4cd7-8bca-08ba63248790"],
        37: ["497d109c-5076-4287-a612-cc9f885150d9"], # 이거 정답 아닐 확률 높음
        29: ["f50cc54c-c51f-45d2-958e-bba326e33417"], # 쓰읍 매우 애매
        102: ["6e3b0253-50a3-4ab5-b387-2af50eccf264"],
        228: ["74a4c893-af24-42d3-b48f-ae6738295200"],
        297: ["bcb32b8a-4ba8-47c6-b565-04569f496a26"],
        278: ["10ad718b-f08c-405c-bffe-b819f64ad012"],
        53: ["34d9ff5e-ccde-4e54-bd7e-e400c925bf62"],
        26: ["3ee43354-16a6-4416-87f5-e380da6dbbe1","7a7412db-e9f4-4797-909b-dc8109011540"],
        265: ["56e5d264-6f82-4e01-8a96-5f369204dc44"],
        216: ["185008d2-6091-4beb-94e4-d847d70fdea6"],
        95: ["647f45f2-bafd-4b6e-a100-2ae559cc8925"],
        305: ["613bcebe-d2b1-4611-9cd5-ac904cd3361f"],
        7: ["281273f5-0adf-4bf7-9b3e-26c77b894ec7","e4d075ab-c60b-4b22-bb31-ea0ca56e1faf"],
        88: ["27422081-5e3e-4716-b7c2-883e7961640d"],
        244: ["fed07bdf-90e8-4236-bc5e-43eb5fe17061"],
        223: ["e2161953-e80b-41b3-a83b-5bc7d76a801c"],
        303: ["ad9423ec-d8cb-4972-b395-871db5cb9f45"],
        286: ["466f5811-9b37-4589-9571-bafadd92c349"],
        266: ["bf023a9d-47f3-4d77-8bf1-5a854b5403a7"],
    }


In [18]:
map = calc_map(gt, pred)
map

0.8920454545454546